In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

sns.set_style('white')

In [ ]:
neighborhoods_df = pd.read_csv('../data/neighborhood_medians.csv')
nyc_df = pd.read_csv('../data/nyc_medians.csv')

neighborhoods_df['Date'] = pd.to_datetime(neighborhoods_df['Date'], format='%Y-%m-%d')
nyc_df['Date'] = pd.to_datetime(nyc_df['Date'], format='%Y-%m-%d')


neighborhoods_df = neighborhoods_df[['Date', 'areaName', 'Median Price']]
nyc_df = nyc_df[['Date', 'Median Price']]

neighborhoods_df = neighborhoods_df.rename(columns={'Date' : 'ds', 'Median Price' : 'y', 'areaName' : 'unique_id'})
nyc_df = nyc_df.rename(columns={'Date' : 'ds', 'Median Price' : 'y'})



In [ ]:
neighborhoods_df.head()

Add COVID exogenous variable.


In [ ]:
sns.lineplot(data=nyc_df, x='ds', y='y')

In [ ]:
neighborhoods_df['during_covid'] = ((neighborhoods_df['ds'] > '2020-03-01') & (neighborhoods_df['ds'] <'2021-12-01')).astype(int)
nyc_df['during_covid'] = ((nyc_df['ds'] > '2020-03-01') & (nyc_df['ds'] <'2021-12-01')).astype(int)


In [ ]:
def split_data(df, test_size=12):
    train_df = df[:-test_size]
    test_df = df[-test_size:]
    
    train_exog = df[:-test_size]['']
    test_exog = df[-test_size:]
    
    return train_df, test_df, train_exog, test_exog

Initial model training.

In [ ]:
from statsforecast import StatsForecast
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
def generate_horizon_exog(df, horizon):
    unique_id = df['unique_id'].unique()[0]
    last_date = df['ds'].max()
    print(last_date)
    forecast_dates = pd.date_range(
        start = last_date + pd.offsets.MonthBegin(1), 
        periods = horizon, 
        freq = 'MS'
    )
    horizon_exog = pd.DataFrame({
        'ds': forecast_dates
    })
    horizon_exog['unique_id'] = unique_id
    horizon_exog['during_covid'] = 0
    
    return horizon_exog

In [ ]:
astoria_exog = generate_horizon_exog(astoria, 12)
astoria_exog

In [ ]:
def split_data(df, test_size = 12, horizon = 12):
    train_df = df[:-test_size][['ds', 'y', 'unique_id', 'during_covid']]
    test_df = df[-test_size:][['ds', 'y', 'unique_id', 'during_covid']]
        
    horizon_exog = generate_horizon_exog(train_df, horizon=horizon)
    
    return train_df, test_df, horizon_exog

In [ ]:
astoria = neighborhoods_df[neighborhoods_df['unique_id'] == "Astoria"]
astoria.head()

In [ ]:
train_df, test_df, horizon_exog = split_data(astoria, horizon = 12)

In [ ]:
sf.plot(train_df)

In [ ]:
a = seasonal_decompose(train_df["y"], model = "add", period=12)
a.plot()

In [ ]:
sns.lineplot(train_df,x="ds", y="y", label="Train")
sns.lineplot(test_df, x="ds", y="y", label="Test")
plt.show()

In [ ]:
from statsforecast.models import AutoARIMA
from statsforecast.arima import arima_string
from statsforecast import StatsForecast

In [ ]:
season_length = 12
horizon = len(test_df)

In [ ]:
sf = StatsForecast(models=[AutoARIMA(season_length=season_length)], freq='MS')
sf.fit(train_df)

In [ ]:
arima_string(sf.fitted_[0,0].model_)

In [ ]:
horizon_exog

In [ ]:
train_df.tail()

In [ ]:
Y_hat_df = sf.forecast(df=train_df, h=horizon, X_df = horizon_exog, fitted=True)
Y_hat_df.head(12)

In [ ]:

def plot_forecasts(train_df, test_df, predictions):
    merged_df = test_df.merge(predictions, how='left', on=['unique_id', 'ds'])
    
    fig, ax = plt.subplots(1, 1, figsize=(18, 7))
    
    plot_df = pd.concat([train_df, predictions]).set_index('ds')
    plot_df[['y', 'AutoARIMA']].plot(ax=ax, linewidth=2)
    
    ax.set_title('Forecast', fontsize=22)
    ax.set_ylabel('Monthly', fontsize=20)
    ax.set_xlabel('Timestamp [t]', fontsize=20)
    ax.legend(prop={'size': 15})
    ax.grid()
    plt.show()

In [ ]:
plot_forecasts(train_df, test_df, Y_hat_df)

In [ ]:
def forecast_neighborhood_exog(train_df, horizon_exog, horizon: 12):
    season_length = 12
    
    sf = StatsForecast(models=[AutoARIMA(season_length=season_length)], freq='MS')
    preds = sf.forecast(df=train_df, X_df = horizon_exog, h=horizon, fitted=True)
    
    return preds

In [ ]:
def forecast_neighborhood(train_df, horizon: 12):
    season_length = 12
    
    sf = StatsForecast(models=[AutoARIMA(season_length=season_length)], freq='MS')
    preds = sf.forecast(df=train_df, h=horizon, fitted=True)
    
    return preds

In [ ]:
astoria_2018 = astoria.query("ds >= '2018-01-01'")
train_df_2018, test_df_2018, horizon_2018 = split_data(astoria_2018, 12, 12)

astoria_forecasts = forecast_neighborhood_exog(train_df_2018, horizon_exog, 12)

In [ ]:
astoria_forecasts

In [ ]:
astoria_2018_no_exog = forecast_neighborhood(train_df_2018[['ds', 'y', 'unique_id']], 12)
astoria_2018_no_exog

In [ ]:
plot_forecasts(train)

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
stats_df = train_df_2018.copy()
stats_df['Date'] = pd.to_datetime(stats_df['ds'])
stats_df = stats_df.set_index('Date')
stats_df = stats_df.asfreq('MS')
stats_df.index.freq = 'MS'


In [ ]:
astoria_model = SARIMAX(train_df_2018['y'], order=(1, 1, 1),
                        seasonal_order=(0, 1, 1, 12))

In [ ]:
result = astoria_model.fit()
print(result.summary())
forecast = result.get_forecast(steps=12)

In [ ]:
last_date = pd.to_datetime(train_df_2018['ds'].iloc[-1])

# Generate forecast with proper dates
forecast = result.get_forecast(steps=12)
forecast_index = pd.date_range(start=last_date + pd.DateOffset(months=1), periods=12, freq='MS')

# Create forecast dataframe
forecast_df = pd.DataFrame({
    'Date': forecast_index,
    'Predicted_Mean': forecast.predicted_mean.values,
    'Lower_CI': forecast.conf_int().iloc[:, 0].values,
    'Upper_CI': forecast.conf_int().iloc[:, 1].values
})

forecast_df

In [ ]:
from itertools import product

best_aic = np.inf
best_params = None

for p, d, q in product(range(3), range(2), range(3)):
    for P, D, Q in product(range(2), range(2), range(2)):
        try:
            model = SARIMAX(train_df_2018['y'],
                           order=(p, d, q),
                           seasonal_order=(P, D, Q, 12))
            result = model.fit(disp=False)
            if result.aic < best_aic:
                best_aic = result.aic
                best_params = ((p, d, q), (P, D, Q, 12))
        except:
            continue

print(f"Best params: {best_params}, AIC: {best_aic}")

Best Params are ((1, 1, 1), (0, 1, 1, 12))

In [ ]:
exog_train = train_df_2018[['during_covid']]
model = SARIMAX(train_df_2018['y'], exog=exog_train, 
                order=(1, 1, 1), seasonal_order=(0, 1, 1, 12))
result = model.fit()

# Forecasting - need future exog values
exog_forecast = test_df_2018['during_covid']  # Must have same columns
forecast = result.get_forecast(steps=12, exog=exog_forecast)

In [ ]:
last_date = pd.to_datetime(train_df_2018['ds'].iloc[-1])

forecast_index = pd.date_range(start=last_date + pd.DateOffset(months=1), periods=12, freq='MS')

# Create forecast dataframe
forecast_df = pd.DataFrame({
    'Date': forecast_index,
    'Predicted_Mean': forecast.predicted_mean.values,
    'Lower_CI': forecast.conf_int().iloc[:, 0].values,
    'Upper_CI': forecast.conf_int().iloc[:, 1].values
})

forecast_df

In [381]:
mae = mean_absolute_error(test_df_2018['y'], forecast_df[['Predicted_Mean']])
print(f"MAE: {mae:.2f}")
rmse = np.sqrt(mean_squared_error(test_df_2018['y'], forecast_df[['Predicted_Mean']]))
print(f"RMSE: {rmse:.2f}")

MAE: 112.80
RMSE: 134.13


After testing both statsforecast AutoARIMA and statsmodels SARIMAX, SARIMAX is the winner with around a 5% error. Now the training loop can be established and all predictions from now to 12 months ahead will be calculated then put in a csv.

In [385]:
neighborhoods = neighborhoods_df['unique_id'].unique()

In [423]:
def train_and_fit_model(train_df, train_exog, steps = 12):
    model = SARIMAX(train_df['y'], exog = train_exog, 
                order = (1, 1, 1), seasonal_order = (0, 1, 1, 12))
    
    result = model.fit()

    exog_forecast = np.zeros((steps, 1))
    forecast = result.get_forecast(steps=steps, exog=exog_forecast)
    
    return forecast, result

In [435]:
def create_forecast_df(train_df, forecast, steps):
    last_date = pd.to_datetime(train_df['ds'].iloc[-1])

    forecast_index = pd.date_range(start=last_date + pd.DateOffset(months=1), periods=steps, freq='MS')

    forecast_df = pd.DataFrame({
        'Date': forecast_index,
        'Predicted_Mean': forecast.predicted_mean.values,
        'Lower_CI': forecast.conf_int().iloc[:, 0].values,
        'Upper_CI': forecast.conf_int().iloc[:, 1].values
    })

    return forecast_df

In [409]:
chinatown = neighborhoods_df.query("unique_id == 'Chinatown'")
chinatown = chinatown[chinatown["ds"] >= '2018-01-01']
chinatown_exog = chinatown['during_covid']

chinatown_forecasts, chinatown_result = train_and_fit_model(chinatown, chinatown_exog, 12)
chinatown_forecast_df = create_forecast_df(chinatown, chinatown_forecasts)

/Users/lukepitstick/nyc-rent-prices/model/.venv/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/Users/lukepitstick/nyc-rent-prices/model/.venv/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/Users/lukepitstick/nyc-rent-prices/model/.venv/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/lukepitstick/nyc-rent-prices/model/.venv/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_mod

In [410]:
chinatown_forecast_df

,Date,Predicted_Mean,Lower_CI,Upper_CI
0,2025-12-01,3972.553212,3492.313258,4452.793165
1,2026-01-01,3917.609546,3371.195144,4464.023948
2,2026-02-01,3827.956262,3248.869296,4407.043229
3,2026-03-01,4040.699128,3437.292309,4644.105947
4,2026-04-01,4125.780573,3500.886994,4750.674152
5,2026-05-01,4185.804166,3540.709806,4830.898526
6,2026-06-01,4255.823858,3591.318148,4920.329568
7,2026-07-01,4123.946627,3440.633929,4807.259326
8,2026-08-01,3988.855723,3287.252395,4690.459050
9,2026-09-01,4045.345299,3325.908214,4764.782384


In [424]:
from tqdm import tqdm

In [439]:
def train_all(neighborhoods, steps):
    for neighborhood in tqdm(neighborhoods, desc="Processing Boroughs"):
        train_df = neighborhoods_df.query(f"unique_id == '{neighborhood}'")
        train_df = train_df[train_df["ds"] >= '2018-01-01']
        train_exog = train_df['during_covid']
        
        forecasts, result = train_and_fit_model(train_df, train_exog, steps)
        forecast_df = create_forecast_df(train_df, forecasts, steps)
        
        try:
            forecast_df.to_csv(f'../output/{neighborhood.replace(" ", "_")}.csv')
        except Exception as e:
            print(forecast_df, e)
        

In [440]:
train_all(neighborhoods, 12)

Processing Boroughs:   0%|          | 0/111 [00:00<?, ?it/s]/Users/lukepitstick/nyc-rent-prices/model/.venv/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/Users/lukepitstick/nyc-rent-prices/model/.venv/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/Users/lukepitstick/nyc-rent-prices/model/.venv/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/lukepitstick/nyc-rent-pric

         Date  Predicted_Mean     Lower_CI     Upper_CI
0  2025-12-01     5565.794209  5116.906060  6014.682357
1  2026-01-01     5846.894352  5318.387215  6375.401490
2  2026-02-01     5694.848203  5056.867532  6332.828873
3  2026-03-01     5663.544373  4950.241066  6376.847680
4  2026-04-01     5775.987901  4986.960352  6565.015450
5  2026-05-01     5873.885066  5019.166150  6728.603983
6  2026-06-01     5835.888409  4918.689571  6753.087247
7  2026-07-01     5835.294942  4860.270867  6810.319017
8  2026-08-01     5797.565096  4767.673949  6827.456243
9  2026-09-01     5742.788634  4660.909507  6824.667762
10 2026-10-01     5777.324262  4645.859069  6908.789454
11 2026-11-01     5795.862584  4616.761967  6974.963201 Cannot save file into a non-existent directory: '../output/Stuyvesant_Town'


/Users/lukepitstick/nyc-rent-prices/model/.venv/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/lukepitstick/nyc-rent-prices/model/.venv/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
Processing Boroughs:  89%|████████▉ | 99/111 [00:21<00:03,  3.55it/s]/Users/lukepitstick/nyc-rent-prices/model/.venv/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/Users/lukepitstick/nyc-rent-prices/mode